# Extract Chapters from consecutive PDFs

In this code I have tried to arrange the PDF files in the an order and then I have tried to extract the Table of contents from the PDF Files. <br>

This code does not do the job of extracting the table of contents properly. The TOC extracted has a lot of content which are not relavant to the context. 

I just realized that we might not even need to worry about making chunks. The documents that are associated with a particular particular project need to be treated as one chunk. We only need to sort these PDF files. 

In [99]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup as bs
import wget

from tika import parser



from pdfminer.pdfparser import PDFParser

from pdfminer.pdfdocument import PDFDocument


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Postal and Int conversion 

### Postal To Int Function 

In [100]:
def postal_to_int(Postal):
    multiplier = 1
    odd = 1
    Postal_Int_Word = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
    
    # Chcek for the length of the Postal Code
    if not(len(Postal)==6):
        return(Error_Message)
    
    for letter in list(str.strip(Postal)):
        if odd%2 == 1:
            # Check if it is an interger or not   
            # If this is not an integer then exit
            if not (str.isnumeric(letter)):
                return(Error_Message)
            
            Postal_Int_Letter = multiplier * int(letter)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 10
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
        else:
            # Check if it is a character or not
            # If this is not a character then exit
            if not (str.isalpha(letter)):
                return(Error_Message)
            
            # ord('A') = 65
            Postal_Int_Letter = multiplier * (ord(letter) - 65)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 26
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
            
    print(1)
    return Postal_Int_Word

### Integer to Postal Function 

In [101]:
def int_to_postal(Postal_Int_Word):
    Postal_Int_Word_Rem = Postal_Int_Word
    Postal = ""
    odd = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
 
    while odd < 6:
        if odd%2 == 0:
            Postal_Int_Letter = Postal_Int_Word_Rem % 10
            Postal = Postal + str(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/10)
            odd = odd + 1

        else:
            Postal_Int_Letter = (Postal_Int_Word_Rem % 26) +65
            Postal = Postal + chr(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/26)
            odd = odd + 1
            
    # Accounts for 7 digit Postal Codes
    if Postal_Int_Word_Rem > 0:
        return(Error_Message)
        
    return Postal[::-1]
  

### Rough Work

In [102]:
Postal = 'A6T6V0'
print(Postal)
print(Postal[::-1])
postal_to_int(Postal)

A6T6V0
0V6T6A
1


456770

In [103]:
int_to_postal(456770)

'A6T6V0'

In [104]:
chr(int(45/10) + 65)

'E'

# Read All PDFs - Full Paths, File Names, Postal, Postal_Int

In [88]:
pdf_path = 'F:/Environmental Baseline Data/Version 2/Data/PDFs'  
text_path = 'F:/Environmental Baseline Data/Version 2/Data/Text'

pdf_paths = os.listdir(pdf_path)

In [89]:
list_pdfs_full_path = [pdf_path +  '/' + str(x) for x in pdf_paths]

list_pdfs_full_path
df_pdfs = pd.DataFrame(list_pdfs_full_path, columns = ['Full_Paths'])

list_file_names = []
TOC_present = []
for full_name in list_pdfs_full_path:
    file_name = full_name.split('/')[-1][:-4]
    
    if 'Thum' not in file_name: # additional check to get rid of the string "Thumb"
        list_file_names.append(file_name)    
    
         
        f = open(text_path +'/' + file_name+'.txt',"r", encoding="utf8")
        if f.mode == 'r':
            contents = f.read()
            contents = contents.upper()
            if 'TABLE OF CONTENTS' in contents:
                TOC_present.append('TOC Present')
            else:
                TOC_present.append('TOC Not Present')
        else:
            TOC_present.append('Error')
            
    else:
        list_file_names.append('Thumb')
        TOC_present.append('Thumb')
    

    
list_file_names
df_pdfs['File_Names'] = list_file_names

postal_series=[]
postal_series = df_pdfs['File_Names'].str.extract('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)')
df_pdfs['Postal'] = postal_series

df_pdfs['TOC_Present'] = TOC_present

['F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C1 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 1.pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C2 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 2 .pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C3 %2D 13.0 EIA %2D Section 13.8 to 13.13.pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D1 V3 %2D ESA_Sec_1%2D11 %2D A4D9H6.pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D10 V4 %2D ESA_Appendix D2 %2D A4D9I5.pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D11 V4 %2D ESA_Appendix D3 %2D A4D9I6.pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D12 V4 %2D ESA_Appendix D4 %2D A4D9I7.pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D13 V4 %2D ESA_Appendix D5 %2D A4D9I8.pdf',
 'F:/Environmental Baseline Data/Version 2

['A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 ',
 'A0H8C1 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 1',
 'A0H8C2 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 2 ',
 'A0H8C3 %2D 13.0 EIA %2D Section 13.8 to 13.13',
 'A63951%2D1 V3 %2D ESA_Sec_1%2D11 %2D A4D9H6',
 'A63951%2D10 V4 %2D ESA_Appendix D2 %2D A4D9I5',
 'A63951%2D11 V4 %2D ESA_Appendix D3 %2D A4D9I6',
 'A63951%2D12 V4 %2D ESA_Appendix D4 %2D A4D9I7',
 'A63951%2D13 V4 %2D ESA_Appendix D5 %2D A4D9I8',
 'A63951%2D14 V4 %2D ESA_Appendix D6 %2D A4D9I9',
 'A63951%2D15 V4 %2D ESA_Appendix D7 %2D A4D9J0',
 'A63951%2D16 V4 %2D ESA_Appendix D8 %2D A4D9J1',
 'A63951%2D17 V4 %2D ESA_Appendix D9 %2D A4D9J2',
 'A63951%2D18 V4 %2D ESA_Appendix D10 %2D A4D9J3',
 'A63951%2D19 V4 %2D ESA_Appendix D11 %2D A4D9J4',
 'A63951%2D2 V4 %2D ESA_Appendix A_EPP_Pipeline %2D A4D9H7',
 'A63951%2D20 V4 %2D ESA_Appendix D12 %2D A4D9J5',
 'A63951%2D21 V4 %2D ESA_Appendix D13 %2D A4D9J6',
 'A63951%2D22 V4 %2D ESA_Appendix D14 %2D A4D9J7',
 'A63951%2D23 V4 %2D 

In [107]:
for postal in postal_series[1:20]:
    print(postal)
type(postal_series)

0


pandas.core.frame.DataFrame

In [97]:
len(df_pdfs)
df_pdfs.head(5)

Postal_Integers = []
for postal in postal_series:
    postal_int = postal_to_int(postal)
    Postal_Integers.append(postal_int)

df_pdfs['Postal_Integers'] = Postal_Integers

# Trial run on a smaller sample  
df_pdfs = df_pdfs[0:200] 
len(df_pdfs)
df_pdfs.head(10)

1322

,Full_Paths,File_Names,Postal,TOC_Present
0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6,A0H8C0,TOC Present
1,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C1 %2D 13.0 EIA %2D Section 13.7 Wildlife ...,A0H8C1,TOC Not Present
2,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C2 %2D 13.0 EIA %2D Section 13.7 Wildlife ...,A0H8C2,TOC Not Present
3,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C3 %2D 13.0 EIA %2D Section 13.8 to 13.13,A0H8C3,TOC Not Present
4,F:/Environmental Baseline Data/Version 2/Data/...,A63951%2D1 V3 %2D ESA_Sec_1%2D11 %2D A4D9H6,A4D9H6,TOC Present


TypeError: 'int' object is not subscriptable

In [87]:
postal_series
postal_int_series = []
for postal in postal_series:
    postal_int = postal_to_int(postal)
    
    if postal_int = 
        
    postal_int_series.append()


,0
0,A0H8C0
1,A0H8C1
2,A0H8C2
3,A0H8C3
4,A4D9H6
5,A4D9I5
6,A4D9I6
7,A4D9I7
8,A4D9I8
9,A4D9I9


In [85]:
df_chapters = pd.DataFrame()
File_Name = []
Chapter_Number =[]
Postal = []
TOC_Present = []


In [86]:
df_pdfs

,Full_Paths,File_Names,Postal,TOC_Present
0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6,A0H8C0,TOC Present
1,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C1 %2D 13.0 EIA %2D Section 13.7 Wildlife ...,A0H8C1,TOC Not Present
2,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C2 %2D 13.0 EIA %2D Section 13.7 Wildlife ...,A0H8C2,TOC Not Present
3,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C3 %2D 13.0 EIA %2D Section 13.8 to 13.13,A0H8C3,TOC Not Present
4,F:/Environmental Baseline Data/Version 2/Data/...,A63951%2D1 V3 %2D ESA_Sec_1%2D11 %2D A4D9H6,A4D9H6,TOC Present
5,F:/Environmental Baseline Data/Version 2/Data/...,A63951%2D10 V4 %2D ESA_Appendix D2 %2D A4D9I5,A4D9I5,TOC Not Present
6,F:/Environmental Baseline Data/Version 2/Data/...,A63951%2D11 V4 %2D ESA_Appendix D3 %2D A4D9I6,A4D9I6,TOC Not Present
7,F:/Environmental Baseline Data/Version 2/Data/...,A63951%2D12 V4 %2D ESA_Appendix D4 %2D A4D9I7,A4D9I7,TOC Not Present
8,F:/Environmental Baseline Data/Version 2/Data/...,A63951%2D13 V4 %2D ESA_Appendix D5 %2D A4D9I8,A4D9I8,TOC Not Present
9,F:/Environmental Baseline Data/Version 2/Data/...,A63951%2D14 V4 %2D ESA_Appendix D6 %2D A4D9I9,A4D9I9,TOC Not Present


In [124]:
# Lets Play from here to 
toc_paths = df_pdfs['Full_Paths'].tolist()
toc_paths = toc_paths[0:2]
toc_paths

file_con = []

['F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf',
 'F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C1 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 1.pdf']

In [144]:
for x in toc_paths:
    fp = open(x, 'rb')
    parser = PDFParser(fp)
    document = PDFDocument(parser)
    try:
        outlines = document.get_outlines()
        for (level,title,dest,a,se) in outlines:
            tocs.append(title)
            file_con.append(title)
            print(title)
        file_con.append(x)
        path_of_toc.append(x) 
        print(x)      
    except: 
        pass
tocs_in_df = ({'path' : path_of_toc, 'toc' : tocs })

Environmental Impact Assessment and Mitigation Plan
Introduction
Project Background
Regulatory Setting




Figure 13.1-1EnCana Ekwan Pipeline
Scope of Environmental Assessment
Project Description
Scope of Project
Pipeline Details
Routing Alternatives




Table 13.2-1Cost Comparisons of the Three Alternatives
Table 13.2-2Linear Corridors Followed by Each Alternative
Pipeline Details
Other Facilities
NGTL Meter Station
Construction
Schedule and Sequencing
Temporary Construction Facilities
Access Requirements
Operational Activities
Assessment Methods
Issues Scoping
Early Public Notification
Traditional Knowledge
Internal Scoping
Defining Impact Parameters
Study Areas
Assessment Scenarios
Identifying Mitigation Options
Quantifying Residual Project Effects
Cumulative Effects Implications




Figure 13.3-1CEA Framework
Table 13.3-1Project Inclusion List
Follow-up and Monitoring
Air
Baseline Setting
Site Description
Climate



Figure 13.4-1Temperature Normals Measured at the Fort Nelson Airpo

Internet Sites
F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf
Environmental Impact Assessment and Mitigation Plan
Introduction
Project Background
Regulatory Setting




Figure 13.1-1EnCana Ekwan Pipeline
Scope of Environmental Assessment
Project Description
Scope of Project
Pipeline Details
Routing Alternatives




Table 13.2-1Cost Comparisons of the Three Alternatives
Table 13.2-2Linear Corridors Followed by Each Alternative
Pipeline Details
Other Facilities
NGTL Meter Station
Construction
Schedule and Sequencing
Temporary Construction Facilities
Access Requirements
Operational Activities
Assessment Methods
Issues Scoping
Early Public Notification
Traditional Knowledge
Internal Scoping
Defining Impact Parameters
Study Areas
Assessment Scenarios
Identifying Mitigation Options
Quantifying Residual Project Effects
Cumulative Effects Implications




Figure 13.3-1CEA Framework
Table 13.3-1Project Inclusion List
Follow-up and Monitoring

EXECUTIVE SUMMARY 
1.0 INTRODUCTION 
1.1 Overview 
1.2 Project Need 
1.3 Regulatory Framework 
Figure 1.1 Regional Location of the Proposed Southern Lights Project 
1.4 Background Information 
 TABLE 1.1  SUMMARY OF RECENT ENBRIDGE PIPELINE APPLICATIONS AND ASSOCIATED SUPPORTING STUDIES BETWEEN CROMER AND GRETNA 
 TABLE 1.2  SUMMARY OF RECENT ENBRIDGE STATION APPLICATIONS AND ASSOCIATED SUPPORTING STUDIES AT STATIONS WITH PROPOSED ADDITIONS / MODIFICATIONS
1.5 Purpose of the Document 
1.6  Project Team 
1.7 Concordance with the NEB Filing Manual 
1.8 References 
 TABLE 1.3  CONCORDANCE WITH GUIDE A – A.2 ENVIRONMENT AND SOCIO-ECONOMIC ASSESSMENT OF THE NEB FILING MANUAL
F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2a Vol II %2D ESA %2D Section 1 %2D Executive Summary and Introduction A0Y1T7.pdf
2.0 PROJECT DESCRIPTION 
2.1 Components of the Project and Location of the Project 
2.1.1 Reversal of Line 13 
TABLE 2.1  PROPOSED MODIFICATIONS AT EXISTING PUMP STATIONS FOR LINE 13 R

5.2 Pump Stations/Terminals 
5.2.1 Edmonton Terminal (SE 5-53-23 W4M and NE 32-52-23 W4M) 
 Figure 5.3 Regional Location of Edmonton Terminal (SE 5-53-23 W4M and NE 32-52-23 W4M) 
5.2.2  Kingman Station (SE 5-49-20 W4M) 
 Figure 5.4 Regional Location of Kingman Station (SE 5-49-20 W4M) 
ESA Vol II - Section 5 (Part 4) - ESE Setting.pdf
5.2.3  Strome Station (SW 2-46-15 W4M) 
 Figure 5.5 Regional Location of Strome Station (SW 2-46-15 W4M) 
5.2.4  Hardisty Terminal (SE 30-42-9 W4M) 
Figure 5.6 Regional Location of the Hardisty Terminal (SE 30-42-9 W4M) 
F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2f  Vol II %2D ESA %2D Section 5 (Part 2) %2D ESE Setting A0Y1U2.pdf
5.2.5  Metiskow Station (SE 1-40-5 W4M)  
Figure 5.7 Regional Location of Metiskow Station (SE 1-40-5 W4M)  
5.2.6  Cactus Lake Station (NE 32-36-27 W3M)  
Figure 5.8 Regional Location of Cactus Lake Station (NE 32-36-27 W3M)  
ESA Vol II - Section 5 (Part 6) - ESE Setting.pdf
5.2.7  Kerrobert Station (SE 34-33-22 W

 TABLE 7.1  POTENTIAL CUMULATIVE RESIDUAL EFFECTS AND MITIGATIVE MEASURES 
 TABLE 7.2  SIGNIFICANCE EVALUATION OF CUMULATIVE EFFECTS OF IDENTIFIED RESIDUAL EFFECTS FOR THE PROPOSED SOUTHERN LIGHTS PROJECT 
 TABLE 7.3  SUMMARY OF CONTACTS REGARDING POTENTIAL CUMULATIVE EFFECTS ON ACCOMMODATION AND MUNICIPAL SERVICES IN SELECTED COMMUNITIES IN THE VICINITY OF THE PROPOSED LSr PIPELINE 
F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2m Vol II %2D ESA %2D Section 7 %2D CEA A0Y1U9.pdf
8.0 INSPECTION, MONITORING AND FOLLOW-UP 
8.1 Environmental Policy 
8.2 Environmental Protection Planning for Construction 
8.2.1 Mitigative Measures 
 Figure 8.1 Enbridge Environmental Policy 
8.2.2 Contingency Plans 
8.2.3 Management Plans 
8.3 Environmental Education during Construction 
8.3.1 Basic Level - Nonsupervisory Construction Personnel and Visitors 
8.3.2 Intermediate Level – Supervisory Construction Personnel and Enbridge Inspectors 
8.3.3 Advanced Level – Enbridge Chief Inspectors and Env

...

Table 8-52 Surface Disturbance for KIs by Physiographic Region
Table 8-52 Surface Disturbance for KIs by Physiographic Region (cont’d)
Mitigation and Effects Management
Old Growth Forests
Rare Plants
Rare Ecological Communities
Wetlands
Riparian Vegetation
Residual Effects
Table 8-53 Residual Effects on Vegetation Diversity from Surface Disturbance in the Eastern Alberta Plains
Table 8-54 Residual Effects on Vegetation Diversity from Surface Disturbance in the Southern Alberta Uplands
Table 8-55 Residual Effects on Vegetation Diversity from Surface Disturbance in the Alberta Plateau
Table 8-56 Residual Effects on Vegetation Diversity from Surface Disturbance in the Rocky Mountains
Table 8-57 Residual Effects on Vegetation Diversity from Surface Disturbance in the Interior Plateau
Table 8-58 Residual Effects on Vegetation Diversity from Surface Disturbance in the Coast Mountains
Table 8-59 Residual Effects on Vegetation Diversity from Surface Disturbance in the Terrestrial PDA
Eastern A

Operations
Table 9-69 Extent of Wetlands in the PEAA and the PDA
Decommissioning
Effects of Habitat Loss: All Phases
Table 9-70 Characterization of the Residual Effects of Change in Habitat Availability on Pond-Dwelling Amphibians
Table 9-70 Characterization of the Residual Effects of Change in Habitat Availability on Pond-Dwelling Amphibians (cont’d)
Cumulative Effects Implications
Table 9-71 Effects on Wildlife from Salvage Harvesting of Pine Stands with Mountain Pine Beetle
Table 9-71 Effects on Wildlife from Salvage Harvesting of Pine Stands with Mountain Pine Beetle (cont’d)
Birds and Mammals
Habitat Loss
Table 9-72 Cumulative Effects on Broad Vegetation Classes in the REAA
Table 9-72 Cumulative Effects on Broad Vegetation Classes in the REAA (cont’d)
Habitat Fragmentation
Table 9-73 Change in Linear Feature Density in the REAA including Future Case
Overall Cumulative Effect
Amphibians
Screening for Cumulative Effects
Prediction Confidence
Table 9-74 Change in Habitat Availability

Table 10C-1 Federal and Provincial Water Quality Guidelines for Human Consumption and for the Protection of Aquatic Life to be Used in Assessing Water Quality (cont’d)
Table 10C-1 Federal and Provincial Water Quality Guidelines for Human Consumption and for the Protection of Aquatic Life to be Used in Assessing Water Quality (cont’d)
Table 10C-1 Federal and Provincial Water Quality Guidelines for Human Consumption and for the Protection of Aquatic Life to be Used in Assessing Water Quality (cont’d)
Table 10C-1 Federal and Provincial Water Quality Guidelines for Human Consumption and for the Protection of Aquatic Life to be Used in Assessing Water Quality (cont’d)
Table 10C-1 Federal and Provincial Water Quality Guidelines for Human Consumption and for the Protection of Aquatic Life to be Used in Assessing Water Quality (cont’d)
Table 10C-1 Federal and Provincial Water Quality Guidelines for Human Consumption and for the Protection of Aquatic Life to be Used in Assessing Water Quality (

Table 11A-1 Pathways of Effects Applicable to the Project
Table 11A-1 Pathways of Effects Applicable to the Project (cont’d)
Table 11A-1 Pathways of Effects Applicable to the Project (cont’d)
Table 11A-1 Pathways of Effects Applicable to the Project (cont’d)
Table 11A-1 Pathways of Effects Applicable to the Project (cont’d)
Table 11A-1 Pathways of Effects Applicable to the Project (cont’d)
Table 11A-1 Pathways of Effects Applicable to the Project (cont’d)
Figure 11A-1 Pathways of Effects Summary – Fisheries and Oceans Canada
Figure 11A-2 Pathways of Effects – Vegetation Clearing
Figure 11A-3 Pathways of Effects – Use of Industrial Equipment
Figure 11A-4 Pathways of Effects – Excavation
Figure 11A-5 Pathways of Effects – Placement of Material or Structures in Water
Figure 11A-6 Pathways of Effects – Change in Timing, Duration and Frequency of Flow
Figure 11A-7 Pathways of Effects – Fish Passage Issues
Figure 11A-8 Pathways of Effects – Grading
Figure 11A-9 Pathways of Effects – Use of E

Palaeontological Resources
Setting for Palaeontological Resources
Table 13-1 Geological Time Scale
Alberta
British Columbia
Foreland Belt
Ominica Belt
Intermontane Belt
Coast-Cascades Belt
Tertiary Deposits
Quaternary Deposits
Scope of Assessment for Palaeontological Resources
Key Project Issues for Palaeontological Resources
Figure 13-1 British Columbia Geological Belts and Terranes
Selection of Valued Environmental Components for Palaeontological Resources
Spatial Boundaries for Palaeontological Resources
Temporal Boundaries for Palaeontological Resources
Regulatory Settings or Administrative Boundaries for Palaeontological Resources
Definition of Environmental Effects Attributes for Palaeontological Resources
Direction
Magnitude
Geographical Extent
Frequency
Duration
Reversibility
Determination of Significance for Palaeontological Resources
General Mitigation Measures for Palaeontological Resources
Palaeontological Sites
Scope of Assessment for Palaeontological Sites
Table 13-2 Pote

In [145]:
type(tocs)

list

In [146]:
contect = ' /n '.join(file_con)
contect

text_file = open("sample.txt", "w")
n = text_file.write(contect)
text_file.close()

"Environmental Impact Assessment and Mitigation Plan /n F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf /n Introduction /n F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf /n Project Background /n F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf /n Regulatory Setting /n F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf /n  /n F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf /n  /n F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf /n  /n F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf /n  /n F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf /n Figure 13.1-1EnCana Ekwan Pipeline /

UnicodeEncodeError: 'charmap' codec can't encode character '\u03bc' in position 390796: character maps to <undefined>

In [ ]:
# playing till here 

In [140]:
# Extracting TOC in PDF 

tocs = []
path_of_toc = []
path_of_toc
toc_paths = df_pdfs['Full_Paths'].tolist()
for x in toc_paths:
    fp = open(x, 'rb')
    parser = PDFParser(fp)
    document = PDFDocument(parser)
    try:
        outlines = document.get_outlines()
        for (level,title,dest,a,se) in outlines:
            tocs.append(title)
        path_of_toc.append(x)            
    except: 
        pass
tocs_in_df = ({'path' : path_of_toc, 'toc' : tocs })

[]

In [141]:
tocs_in_df

{'path': ['F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf',
  'F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C1 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 1.pdf',
  'F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2a Vol II %2D ESA %2D Section 1 %2D Executive Summary and Introduction A0Y1T7.pdf',
  'F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2b Vol II %2D ESA %2D Section 2 %2D Project Description A0Y1T8.pdf',
  'F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2c Vol II %2D ESA %2D Section 3 %2D Consultation A0Y1T9.pdf',
  'F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2d Vol II %2D ESA %2D Section 4 %2D Routing A0Y1U0.pdf',
  'F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2e Vol II %2D ESA %2D Section 5 (Part 1) %2D ESE Setting A0Y1U1.pdf',
  'F:/Environmental Baseline Data/Version 2/Data/PDFs/B%2D2f  Vol II %2D ESA %2D Section 5 (Part 2) %2D ESE Setting A0Y1U2.pdf',
  'F:/Enviro

In [142]:
len(df_pdfs)

200

In [143]:
df_pdfs = pd.DataFrame(path_of_toc, tocs, columns = ['Full_Paths', 'TOC'])
df_pdfs.head(5)

ValueError: Shape of passed values is (118, 1), indices imply (4640, 2)

In [61]:
f = open(text_path +'/' +df_pdfs.File_Names.values.tolist()[1]+'.txt',"r", encoding="utf8")
if f.mode == 'r':
    contents = f.read()
    
contents


"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMicrosoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc\n\n\n \n\nENVIRONMENTAL IMPACT ASSESSMENT\nAND MITIGATION PLAN\n\n Page 13-150\n \n \n\n \nEnCana Ekwan Pipeline Inc. \n\n13.8 Fisheries and Aquatic Resources \n\n13.8.1 Baseline Setting \n\n13.8.1.1 Fish Species Present \n\nThe proposed 83 km-long Project is located entirely within a portion of the Hay River \nwatershed (Fort Nelson Lowlands) that is characterized by poorly drained muskeg \nlowlands, providing a mosaic of forest and wetland ecosystems (Meidinger and Pojar 1991; \nAppendix C). Streams generally have low fish habitat diversity and capacity. Channels are \ninvariably sinuous, low gradient (<1 percent) with substrates dominated by fines.  \n\nThe Hay River rises in the Naylor Hills of northwestern Alberta, flows westward into British \nColumbia before turning northeast back into Alberta, then north into the Northwest \nTerritories and 

In [66]:
contents = contents.upper()
'STIZOSTED' in contents

True

In [ ]:
raw = parser.from_file(df_pdfs.Full_Paths.values.tolist()[1])
raw['content']

